In [1]:
# Core scientific stack
import os
import numpy as np
import pandas as pd

# SunPy + JSOC
import sunpy.map
from sunpy.net import Fido, attrs as a

# FITS handling
from astropy.io import fits
from astropy.time import Time

# Progress
from tqdm import tqdm

# Plotting (optional sanity checks)
import matplotlib.pyplot as plt


C:\Users\Owner\anaconda3\envs\solar_flare_one\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load SHARP metadata
SHARP_META = "data/sharp_metadata_labeled.parquet"
sharp = pd.read_parquet(SHARP_META)

print("Total SHARP rows:", len(sharp))
sharp.head()


Total SHARP rows: 2613618


,T_REC,HARPNUM,NOAA_ARS,USFLUX,R_VALUE,MEANGAM,MEANGBT,MEANGBZ,TOTUSJZ,TOTUSJH,AREA_ACR,QUALITY,label_MX_24h
0,2010-05-01 00:12:00,1,11067,6.521054e+21,3.642,29.678,68.349,90.781,5.654726e+12,264.299,105.709244,0,0
1,2010-05-01 00:24:00,1,11067,6.917875e+21,3.357,28.441,67.682,89.127,6.488687e+12,307.572,111.348442,0,0
2,2010-05-01 00:36:00,1,11067,6.973706e+21,3.544,28.031,67.166,85.321,6.193157e+12,297.073,108.848373,0,0
3,2010-05-01 00:48:00,1,11067,7.228647e+21,3.073,26.980,64.805,76.349,5.797055e+12,301.562,104.785210,0,0
4,2010-05-01 01:00:00,1,11067,7.361199e+21,3.332,26.925,66.337,80.902,6.081439e+12,316.165,106.967400,0,0


In [4]:
# Cell 3 — Filter SHARP rows and ensure T_UTC is datetime (FIXED)

# Bobra quality filter
sharp = sharp[sharp['QUALITY'] < 65536].copy()

# T_REC is already datetime when loaded from parquet
# Just rename / copy it safely
sharp['T_UTC'] = pd.to_datetime(sharp['T_REC'], errors='coerce')

# Drop invalid times
sharp = sharp.dropna(subset=['T_UTC'])

print("Usable SHARP rows:", len(sharp))
sharp[['HARPNUM', 'T_REC', 'T_UTC']].head()


Usable SHARP rows: 2613618


,HARPNUM,T_REC,T_UTC
0,1,2010-05-01 00:12:00,2010-05-01 00:12:00
1,1,2010-05-01 00:24:00,2010-05-01 00:24:00
2,1,2010-05-01 00:36:00,2010-05-01 00:36:00
3,1,2010-05-01 00:48:00,2010-05-01 00:48:00
4,1,2010-05-01 01:00:00,2010-05-01 01:00:00


In [5]:
# SHARP parameters used for CNN input
CNN_CHANNELS = [
    'USFLUX',
    'MEANGAM',
    'MEANGBT',
    'MEANGBZ',
    'MEANGBH',
    'MEANJZD',
    'TOTUSJZ',
    'MEANALP',
    'MEANJZH',
    'TOTUSJH'
]

print("CNN channels:", CNN_CHANNELS)


CNN channels: ['USFLUX', 'MEANGAM', 'MEANGBT', 'MEANGBZ', 'MEANGBH', 'MEANJZD', 'TOTUSJZ', 'MEANALP', 'MEANJZH', 'TOTUSJH']


In [6]:
# Output root directory
OUT_ROOT = "cnn_data"
os.makedirs(OUT_ROOT, exist_ok=True)

for ch in CNN_CHANNELS:
    os.makedirs(os.path.join(OUT_ROOT, ch), exist_ok=True)

print("Output directories created.")


Output directories created.


In [11]:
sharp = sharp.sort_values(["HARPNUM", "T_UTC"]).copy()

sharp["DATE_UTC"] = sharp["T_UTC"].dt.date

sharp = (
    sharp
    .groupby(["HARPNUM", "DATE_UTC"], as_index=False)
    .head(1)
    .reset_index(drop=True)
)

print("Rows after Bobra-style daily subsampling:", len(sharp))
sharp[["HARPNUM", "T_UTC", "DATE_UTC"]].head()


Rows after Bobra-style daily subsampling: 28740


,HARPNUM,T_UTC,DATE_UTC
0,1,2010-05-01 00:12:00,2010-05-01
1,1,2010-05-02 00:00:00,2010-05-02
2,1,2010-05-03 00:12:00,2010-05-03
3,1,2010-05-04 00:12:00,2010-05-04
4,1,2010-05-05 00:00:00,2010-05-05


In [1]:
#START WITH NEW EXTRACTION
import pandas as pd
import numpy as np
from datetime import timedelta


In [2]:
# CNN sampling cadence
CNN_CADENCE_HOURS = 2

# Temporal context for CNN (match LSTM lookback)
LOOKBACK_HOURS = 6

# JSOC series
JSOC_SERIES = "hmi.sharp_cea_720s"

# CNN image channels
CNN_SEGMENTS = ["Br", "Bp", "Bt"]


In [8]:
flare_df = pd.read_csv("hek_flares_chunked.csv")

print("Columns in hek_flares_chunked.csv:")
print(flare_df.columns.tolist())


C:\Users\Owner\AppData\Local\Temp\ipykernel_20128\1343333238.py:1: DtypeWarning: Columns (22,105,128,134) have mixed types. Specify dtype option on import or set low_memory=False.
  flare_df = pd.read_csv("hek_flares_chunked.csv")


Columns in hek_flares_chunked.csv:
['gs_thumburl', 'comment_count', 'hpc_bbox', 'frm_humanflag', 'hgc_coord', 'event_coordsys', 'obs_levelnum', 'hpc_coord', 'event_npixels', 'gs_imageurl', 'ar_polarity', 'frm_paramset', 'hrc_coord', 'event_starttime', 'ar_mtwilsoncls', 'event_type', 'intensmin', 'fl_fluence', 'obs_meanwavel', 'frm_url', 'skel_chaincode', 'bound_chaincode', 'noposition', 'fl_fluenceunit', 'active', 'intensmax', 'frm_versionnumber', 'fl_peaktempunit', 'fl_halphaclass', 'area_uncert', 'obs_dataprepurl', 'hpc_geom', 'hgc_bbox', 'intensmedian', 'chaincodetype', 'obs_channelid', 'event_clippedspatial', 'ar_noaaclass', 'SOL_standard', 'event_avg_rating', 'eventtype', 'intensunit', 'hpc_boundcc', 'event_mapurl', 'frm_contact', 'ar_penumbracls', 'intensmean', 'bound_ccstartc1', 'frm_name', 'area_atdiskcenter', 'frm_identifier', 'obs_observatory', 'event_description', 'boundbox_c2ur', 'obs_firstprocessingdate', 'boundbox_c2ll', 'frm_institute', 'hrc_bbox', 'refs_orig', 'ar_mcint

In [9]:
flare_df = flare_df.rename(columns={
    "event_starttime": "flare_time",
    "ar_noaanum": "noaa_ar"
})

flare_df["flare_time"] = pd.to_datetime(flare_df["flare_time"])


In [10]:
print("Sample flare rows:")
print(flare_df[["flare_time", "noaa_ar"]].head())

print("\nData types:")
print(flare_df[["flare_time", "noaa_ar"]].dtypes)


Sample flare rows:
           flare_time  noaa_ar
0 2010-01-01 12:02:00   1039.0
1 2010-01-01 12:33:00   1039.0
2 2010-01-01 23:29:00      0.0
3 2010-01-02 03:10:00  11039.0
4 2010-01-02 07:09:00  11039.0

Data types:
flare_time    datetime64[ns]
noaa_ar              float64
dtype: object


In [11]:
print("Total number of flare events:", len(flare_df))


Total number of flare events: 119306


In [1]:
import os
import glob
import pandas as pd
import numpy as np
from datetime import timedelta

print("Working directory:", os.getcwd())


Working directory: C:\Users\Owner\OneDrive\Desktop\solar_proj


In [2]:
CNN_CADENCE_HOURS = 2
LOOKBACK_HOURS = 6

JSOC_SERIES = "hmi.sharp_cea_720s"
CNN_SEGMENTS = ["Br", "Bp", "Bt"]


In [4]:
flare_df = pd.read_csv("hek_flares_chunked.csv", low_memory=False)

flare_df = flare_df.rename(columns={
    "event_starttime": "flare_time",
    "ar_noaanum": "noaa_ar"
})

flare_df["flare_time"] = pd.to_datetime(flare_df["flare_time"])

# 🔑 CRITICAL: drop invalid NOAA ARs BEFORE casting
flare_df = flare_df.dropna(subset=["noaa_ar"])
flare_df = flare_df[flare_df["noaa_ar"] > 0]

flare_df["noaa_ar"] = flare_df["noaa_ar"].astype(int)

print("Clean flare rows:", len(flare_df))
print(flare_df[["flare_time", "noaa_ar"]].head())


Clean flare rows: 15505
           flare_time  noaa_ar
0 2010-01-01 12:02:00     1039
1 2010-01-01 12:33:00     1039
3 2010-01-02 03:10:00    11039
4 2010-01-02 07:09:00    11039
5 2010-01-02 07:58:00     1039


In [5]:
mapping_files = sorted(glob.glob("mharp_metadata/mharp_*.csv"))

print("Mapping files found:", len(mapping_files))
print("Sample files:", mapping_files[:5])


Mapping files found: 81
Sample files: ['mharp_metadata\\mharp_2010_04.csv', 'mharp_metadata\\mharp_2010_05.csv', 'mharp_metadata\\mharp_2010_06.csv', 'mharp_metadata\\mharp_2010_07.csv', 'mharp_metadata\\mharp_2010_08.csv']


In [6]:
dfs = [pd.read_csv(f) for f in mapping_files]
harp_map = pd.concat(dfs, ignore_index=True)

harp_map = harp_map.rename(columns={
    "NOAA_AR": "noaa_ar",
    "HARPNUM": "harpnum"
})

harp_map = harp_map.dropna(subset=["noaa_ar", "harpnum"])
harp_map["noaa_ar"] = harp_map["noaa_ar"].astype(int)
harp_map["harpnum"] = harp_map["harpnum"].astype(int)

harp_map = harp_map.drop_duplicates(subset=["noaa_ar", "harpnum"])

print("Clean mapping rows:", len(harp_map))
print(harp_map.head())


Clean mapping rows: 1579
      harpnum  noaa_ar                T_REC
0           1    11067           2010-05-01
1           2    11064           2010-05-01
2           5        0           2010-05-01
3           6    11065           2010-05-01
3088        8    11069  2010-05-01 14:36:00


In [7]:
flare_df = flare_df.merge(
    harp_map[["noaa_ar", "harpnum"]],
    on="noaa_ar",
    how="inner"
)

print("Flares with HARPNUM:", len(flare_df))
print(flare_df[["flare_time", "noaa_ar", "harpnum"]].head())


Flares with HARPNUM: 2230
           flare_time  noaa_ar  harpnum
0 2010-05-01 05:23:00    11064        2
1 2010-05-05 00:55:00    11069        8
2 2010-05-05 08:35:00    11069        8
3 2010-05-05 13:00:00    11069        8
4 2010-05-05 16:00:02    11069        8


In [27]:
VALID_MINUTES = np.array([0, 12, 24, 36, 48])

def snap_to_hmi(t):
    diffs = np.abs(VALID_MINUTES - t.minute)
    nearest_minute = VALID_MINUTES[np.argmin(diffs)]
    return t.replace(minute=int(nearest_minute), second=0)

def generate_cnn_times(flare_time):
    start = flare_time - timedelta(hours=LOOKBACK_HOURS)
    raw_times = pd.date_range(
        start=start,
        end=flare_time,
        freq=f"{CNN_CADENCE_HOURS}H",
        inclusive="left"
    )
    return [snap_to_hmi(t) for t in raw_times]

flare_df["cnn_times"] = flare_df["flare_time"].apply(generate_cnn_times)


C:\Users\Owner\AppData\Local\Temp\ipykernel_43328\760728289.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  raw_times = pd.date_range(


In [28]:
records = []

for _, row in flare_df.iterrows():
    for t in row["cnn_times"]:
        records.append({
            "harpnum": row["harpnum"],
            "t_rec": t.strftime("%Y.%m.%d_%H:%M:%S_TAI")
        })

cnn_extract_df = pd.DataFrame(records)

print("Total CNN images to extract:", len(cnn_extract_df))
print("Median images per region:",
      cnn_extract_df.groupby("harpnum").size().median())

cnn_extract_df.head()


Total CNN images to extract: 6690
Median images per region: 12.0


,harpnum,t_rec
0,2,2010.04.30_23:24:00_TAI
1,2,2010.05.01_01:24:00_TAI
2,2,2010.05.01_03:24:00_TAI
3,8,2010.05.04_18:48:00_TAI
4,8,2010.05.04_20:48:00_TAI


In [29]:
def build_jsoc_query(harpnum, t_rec):
    return (
        f"{JSOC_SERIES}[{harpnum}][{t_rec}]"
        f"{{{','.join(CNN_SEGMENTS)}}}"
    )

cnn_extract_df["jsoc_query"] = cnn_extract_df.apply(
    lambda r: build_jsoc_query(r["harpnum"], r["t_rec"]),
    axis=1
)

print("Sample JSOC query:")
print(cnn_extract_df["jsoc_query"].iloc[0])


Sample JSOC query:
hmi.sharp_cea_720s[2][2010.04.30_23:24:00_TAI]{Br,Bp,Bt}


In [30]:
cnn_extract_df.to_csv(
    "cnn_extraction_plan_2h_6hlookback.csv",
    index=False
)

print("Extraction plan saved.")


Extraction plan saved.


In [31]:
print("Total CNN images planned:", len(cnn_extract_df))
print("Unique HARPNUMs:", cnn_extract_df["harpnum"].nunique())
print("Median images per flare:",
      cnn_extract_df.groupby("harpnum").size().median())


Total CNN images planned: 6690
Unique HARPNUMs: 314
Median images per flare: 12.0


In [32]:
BATCH_SIZE = 100

batches = [
    cnn_extract_df.iloc[i:i+BATCH_SIZE]
    for i in range(0, len(cnn_extract_df), BATCH_SIZE)
]

print("Total batches:", len(batches))
print("First batch size:", len(batches[0]))


Total batches: 67
First batch size: 100


In [33]:
import os

os.makedirs("jsoc_batches", exist_ok=True)

for i, batch in enumerate(batches):
    batch_file = f"jsoc_batches/cnn_batch_{i:04d}.txt"
    batch["jsoc_query"].to_csv(
        batch_file,
        index=False,
        header=False
    )

print("JSOC batch files written.")


JSOC batch files written.


In [36]:
from sunpy.net import Fido, attrs as a
import time

# 1. Put your timestamps into a clean Python list
# (I've truncated this for the example, but use your full list here)
record_list = [
    "hmi.sharp_cea_720s[2][2010.04.30_23:24:00_TAI]{Br,Bp,Bt}",
    "hmi.sharp_cea_720s[2][2010.05.01_01:24:00_TAI]{Br,Bp,Bt}",
    "hmi.sharp_cea_720s[2][2010.05.01_03:24:00_TAI]{Br,Bp,Bt}",
    # ... add all 100 here ...
]

# 2. Iterate and Search
# We'll collect the results in a list
all_results = []

print(f"Starting search for {len(record_list)} records...")

for rec in record_list:
    try:
        res = Fido.search(
            a.jsoc.RecordSet(rec),
            a.jsoc.Notify("vishwastattala@gmail.com")
        )
        all_results.append(res)
    except Exception as e:
        print(f"Failed to parse record: {rec}. Error: {e}")

# 3. Fetch (Download)
# This will download everything that was successfully found
for result in all_results:
    if result:
        Fido.fetch(result)
        # Small sleep to prevent overwhelming the JSOC export queue
        time.sleep(0.5)

Starting search for 3 records...
Failed to parse record: hmi.sharp_cea_720s[2][2010.04.30_23:24:00_TAI]{Br,Bp,Bt}. Error: module 'sunpy.net.jsoc.attrs' has no attribute 'RecordSet'
Failed to parse record: hmi.sharp_cea_720s[2][2010.05.01_01:24:00_TAI]{Br,Bp,Bt}. Error: module 'sunpy.net.jsoc.attrs' has no attribute 'RecordSet'
Failed to parse record: hmi.sharp_cea_720s[2][2010.05.01_03:24:00_TAI]{Br,Bp,Bt}. Error: module 'sunpy.net.jsoc.attrs' has no attribute 'RecordSet'


In [40]:
# Robust cleaning + batching for JSOC recordsets (no regex)
import os
from datetime import datetime
import pandas as pd

# Load DataFrame if not in memory
try:
    cnn_extract_df
except NameError:
    cnn_extract_df = pd.read_csv("cnn_extraction_plan_2h_6hlookback.csv")

os.makedirs("jsoc_batches", exist_ok=True)

def remove_hidden_and_trim(s):
    # strip quotes and whitespace, remove common zero-width/BOM characters
    s = s.strip().strip('"').strip("'")
    s = ''.join(ch for ch in s if ord(ch) >= 32 and ch not in ("\u200B","\u200C","\u200D","\uFEFF"))
    return s.replace(" ", "")

def extract_timestamp_token(rs):
    # Find the second '[' ... ']' token which contains the timestamp
    # Return the raw token string (including _TAI) or None
    opens = [i for i, ch in enumerate(rs) if ch == "["]
    closes = [i for i, ch in enumerate(rs) if ch == "]"]
    if len(opens) < 2 or len(closes) < 2:
        return None
    # match the second pair by order
    start = opens[1] + 1
    # find the first close after that start
    end = next((c for c in closes if c > opens[1]), None)
    if end is None:
        return None
    return rs[start:end]

def validate_timestamp_token(tok):
    # tok should look like YYYY.MM.DD_HH:MM:SS_TAI
    if tok is None:
        return False
    try:
        dt = datetime.strptime(tok, "%Y.%m.%d_%H:%M:%S_TAI")
    except Exception:
        return False
    return dt.second == 0 and dt.minute in {0, 12, 24, 36, 48}

# Build cleaned, deduplicated list preserving order
raw = cnn_extract_df["jsoc_query"].astype(str).tolist()
seen = set()
unique = []
for r in raw:
    c = remove_hidden_and_trim(r)
    if c not in seen:
        seen.add(c)
        unique.append(c)

valid = []
invalid = []
for r in unique:
    tok = extract_timestamp_token(r)
    if validate_timestamp_token(tok):
        valid.append(r)
    else:
        invalid.append(r)

print("Unique recordsets:", len(unique))
print("Valid recordsets:", len(valid))
print("Invalid recordsets:", len(invalid))
if invalid:
    print("First invalid examples (up to 10):")
    for x in invalid[:10]:
        print("  INVALID:", x)

# Write full cleaned files (newline and comma)
with open("jsoc_batch_newline.txt", "w", newline="\n") as f:
    f.write("\n".join(valid))
with open("jsoc_batch_comma.txt", "w", newline="\n") as f:
    f.write(",".join(valid))

# Also write smaller batches to avoid paste/length limits
BATCH_SIZE = 50
for i in range(0, len(valid), BATCH_SIZE):
    chunk = valid[i:i+BATCH_SIZE]
    idx = i // BATCH_SIZE
    with open(f"jsoc_batches/cnn_batch_{idx:03d}_comma.txt", "w") as f:
        f.write(",".join(chunk))
    with open(f"jsoc_batches/cnn_batch_{idx:03d}_newline.txt", "w") as f:
        f.write("\n".join(chunk))

print("Wrote cleaned batches to jsoc_batches/ and jsoc_batch_*.txt")
print("Example first 3 valid recordsets:")
for r in valid[:3]:
    print(" ", r)


Unique recordsets: 5240
Valid recordsets: 5240
Invalid recordsets: 0
Wrote cleaned batches to jsoc_batches/ and jsoc_batch_*.txt
Example first 3 valid recordsets:
  hmi.sharp_cea_720s[2][2010.04.30_23:24:00_TAI]{Br,Bp,Bt}
  hmi.sharp_cea_720s[2][2010.05.01_01:24:00_TAI]{Br,Bp,Bt}
  hmi.sharp_cea_720s[2][2010.05.01_03:24:00_TAI]{Br,Bp,Bt}


In [ ]:
# Fixed batch parser + SunPy submission (handles commas inside {Br,Bp,Bt})
import os
import astropy.units as u
from sunpy.net import Fido, attrs as a

# --- CONFIGURE ---
JSOC_EMAIL = "vishwastattala@gmail.com"  # set this
BATCH_DIR = "jsoc_batches"                              # cleaned batches
SERIES = "hmi.sharp_cea_720s"
SEGMENTS = ["Br", "Bp", "Bt"]                          # segments you want
# -----------------

def split_recordsets_preserve_braces(s):
    """Split a single-line or multi-line batch string into recordset strings.
    Splits only at recordset boundaries (a closing '}' ends a recordset).
    Returns list of trimmed recordset strings.
    """
    out = []
    buf = []
    for ch in s:
        buf.append(ch)
        if ch == '}':
            token = ''.join(buf).strip()
            if token:
                out.append(token)
            buf = []
        # continue accumulating until next closing brace
    # if anything left in buffer, try to add trimmed remainder
    remainder = ''.join(buf).strip()
    if remainder:
        out.append(remainder)
    # remove any trailing commas or empty tokens
    cleaned = []
    for r in out:
        r = r.strip()
        # strip trailing commas that might remain
        if r.endswith(','):
            r = r[:-1].strip()
        if r:
            cleaned.append(r)
    return cleaned

def parse_recordset(rs):
    # Expect: hmi.sharp_cea_720s[HARPNUM][YYYY.MM.DD_HH:MM:SS_TAI]{Br,Bp,Bt}
    rs = rs.strip()
    if not rs:
        return None, None
    try:
        parts = rs.split('[')
        harp = parts[1].split(']')[0]
        t_rec = parts[2].split(']')[0]
        return int(harp), t_rec
    except Exception:
        return None, None

def submit_batch(recs, batch_name):
    searches = []
    for r in recs:
        harp, t_rec = parse_recordset(r)
        if harp is None:
            print(f"Skipping malformed record: {r}")
            continue
        # convert T_REC to ISO-like string for a.Time: "YYYY.MM.DD_HH:MM:SS_TAI" -> "YYYY-MM-DD HH:MM:SS"
        t_iso = t_rec.replace("_TAI", "").replace(".", "-").replace("_", " ")
        attrs = [
            a.Time(t_iso, t_iso),
            a.Sample(12 * u.minute),
            a.jsoc.Series(SERIES),
            a.jsoc.PrimeKey("HARPNUM", harp),
            a.jsoc.Notify(JSOC_EMAIL)
        ]
        # Add one Segment attr per segment
        for seg in SEGMENTS:
            attrs.append(a.jsoc.Segment(seg))
        try:
            res = Fido.search(*attrs)
            searches.append(res)
        except Exception as e:
            print("Search failed for", r, ":", repr(e))
    if not searches:
        print("No valid searches in batch", batch_name)
        return
    combined = searches[0]
    for s in searches[1:]:
        combined = combined + s
    try:
        files = Fido.fetch(combined)
        print(f"Batch {batch_name}: downloaded {len(files)} files")
    except Exception as e:
        print(f"Batch {batch_name} fetch failed:", repr(e))

# Iterate cleaned batch files
if not os.path.isdir(BATCH_DIR):
    raise SystemExit(f"Batch directory not found: {BATCH_DIR}")

for fname in sorted(os.listdir(BATCH_DIR)):
    if not (fname.endswith("_comma.txt") or fname.endswith("_newline.txt")):
        continue
    path = os.path.join(BATCH_DIR, fname)
    with open(path, "r") as f:
        content = f.read().strip()
    # Use robust splitter that preserves commas inside braces
    recs = split_recordsets_preserve_braces(content)
    recs = [r.strip().rstrip(',') for r in recs if r.strip()]
    print("Submitting batch:", fname, "records:", len(recs))
    submit_batch(recs, fname)


In [43]:
with open("jsoc_batches/cnn_batch_000_comma.txt") as f:
    s = f.read().strip()
print("Batch length (chars):", len(s))
print("Count of '}' in file:", s.count("}"))


Batch length (chars): 2878
Count of '}' in file: 50


In [44]:
from pathlib import Path
s = Path("jsoc_batches/cnn_batch_000_comma.txt").read_text()
def split_recordsets_preserve_braces(s):
    out, buf = [], []
    for ch in s:
        buf.append(ch)
        if ch == '}':
            out.append(''.join(buf).strip())
            buf = []
    if buf:
        rem = ''.join(buf).strip()
        if rem: out.append(rem)
    return [r.rstrip(',').strip() for r in out if r.strip()]
print("Parsed recordsets:", len(split_recordsets_preserve_braces(s)))
print("First 3 parsed:", split_recordsets_preserve_braces(s)[:3])


Parsed recordsets: 50
First 3 parsed: ['hmi.sharp_cea_720s[2][2010.04.30_23:24:00_TAI]{Br,Bp,Bt}', ',hmi.sharp_cea_720s[2][2010.05.01_01:24:00_TAI]{Br,Bp,Bt}', ',hmi.sharp_cea_720s[2][2010.05.01_03:24:00_TAI]{Br,Bp,Bt}']


In [49]:
from pathlib import Path

total = 0

for p in Path("jsoc_batches").glob("*_newline.txt"):
    s = p.read_text()
    total += s.count("}")

print("TOTAL refined timestamps across ALL batches:", total)


TOTAL refined timestamps across ALL batches: 5240


In [63]:
# newline_audit_summary.py
# Run in project root where jsoc_audit/ and jsoc_batches/ exist.
import json
from pathlib import Path
from collections import Counter

AUDIT_DIR = Path("jsoc_audit")
BATCH_DIR = Path("jsoc_batches")
SEGMENTS_PER_RECORD = 3  # Br, Bp, Bt

# Find newline audit files
newline_audits = sorted(AUDIT_DIR.glob("*_newline_audit.json"))
if not newline_audits:
    raise SystemExit("No *_newline_audit.json files found in jsoc_audit/")

total_batches = len(newline_audits)
total_recordsets = 0
successful_recordsets = 0
failed_recordsets = 0
failed_by_batch = Counter()
all_file_paths = set()

for a in newline_audits:
    try:
        d = json.loads(a.read_text())
    except Exception as e:
        print("Failed to read", a.name, ":", e)
        continue
    total_recordsets += len(d)
    for rec, val in d.items():
        if val and isinstance(val, list) and len(val) > 0:
            successful_recordsets += 1
            for p in val:
                all_file_paths.add(p)
        else:
            failed_recordsets += 1
            failed_by_batch[a.name] += 1

# Files on disk check
files_on_disk = 0
files_missing = 0
missing_examples = []
for p in sorted(all_file_paths):
    if Path(p).exists():
        files_on_disk += 1
    else:
        files_missing += 1
        if len(missing_examples) < 10:
            missing_examples.append(p)

# Expected files (assuming each successful recordset should produce 3 segment files)
expected_files = successful_recordsets * SEGMENTS_PER_RECORD
remaining_to_fetch = expected_files - files_on_disk

# Print summary
print("\n=== NEWLINE AUDIT SUMMARY ===")
print(f"Newline audit files scanned: {total_batches}")
print(f"Total recordsets requested: {total_recordsets}")
print(f"Successful recordsets: {successful_recordsets}")
print(f"Failed recordsets: {failed_recordsets}")
print(f"Unique FITS file paths listed in audits: {len(all_file_paths)}")
print(f"Files present on disk: {files_on_disk}")
print(f"Files missing on disk: {files_missing}")
print(f"Expected FITS files (successes x {SEGMENTS_PER_RECORD}): {expected_files}")
print(f"Remaining files to fetch to reach expected: {remaining_to_fetch}")
print("\nTop 10 batches by failed recordsets:")
for name, cnt in failed_by_batch.most_common(10):
    print(f"  {name}: {cnt} failed")

if missing_examples:
    print("\nExamples of missing file paths (up to 10):")
    for p in missing_examples:
        print(" ", p)

# Write a small CSV-like summary for quick inspection
OUT = Path("newline_audit_summary.txt")
with OUT.open("w", encoding="utf-8") as f:
    f.write("newline_batches_scanned,total_recordsets,successful,failed,unique_file_paths,files_on_disk,files_missing,expected_files,remaining_to_fetch\n")
    f.write(f"{total_batches},{total_recordsets},{successful_recordsets},{failed_recordsets},{len(all_file_paths)},{files_on_disk},{files_missing},{expected_files},{remaining_to_fetch}\n")
print("\nWrote concise summary to", OUT.resolve())



=== NEWLINE AUDIT SUMMARY ===
Newline audit files scanned: 105
Total recordsets requested: 5240
Successful recordsets: 2991
Failed recordsets: 2249
Unique FITS file paths listed in audits: 8972
Files present on disk: 8972
Files missing on disk: 0
Expected FITS files (successes x 3): 8973
Remaining files to fetch to reach expected: 1

Top 10 batches by failed recordsets:
  cnn_batch_065_newline_audit.json: 50 failed
  cnn_batch_066_newline_audit.json: 50 failed
  cnn_batch_067_newline_audit.json: 50 failed
  cnn_batch_068_newline_audit.json: 50 failed
  cnn_batch_069_newline_audit.json: 50 failed
  cnn_batch_070_newline_audit.json: 50 failed
  cnn_batch_071_newline_audit.json: 50 failed
  cnn_batch_072_newline_audit.json: 50 failed
  cnn_batch_073_newline_audit.json: 50 failed
  cnn_batch_074_newline_audit.json: 50 failed

Wrote concise summary to C:\Users\Owner\OneDrive\Desktop\solar_proj\newline_audit_summary.txt


In [67]:
import json
from pathlib import Path

AUDIT_DIR = Path("jsoc_audit")
SEGMENTS_PER_RECORD = 3

# count successful recordsets from newline audits
success = 0
for a in AUDIT_DIR.glob("*_newline_audit.json"):
    d = json.loads(a.read_text())
    success += sum(1 for v in d.values() if v and isinstance(v, list) and len(v) > 0)

expected_files = success * SEGMENTS_PER_RECORD

# count actual .fits files under project root
root = Path('.')
actual_files = sum(1 for _ in root.rglob('*.fits'))

print("Successful recordsets:", success)
print("Expected FITS files:", expected_files)
print("Actual FITS files found:", actual_files)
print("Shortfall (expected - actual):", expected_files - actual_files)


Successful recordsets: 2991
Expected FITS files: 8973
Actual FITS files found: 17514
Shortfall (expected - actual): -8541


In [85]:
from sunpy import config
print(config.get('downloads', 'download_dir'))


C:/Users/Owner/sunpy/data


In [97]:
from pathlib import Path
from astropy.io import fits
import numpy as np

folder = Path(r"C:/Users/Owner/sunpy/data")
fits_file = next(folder.glob("*.fits"))

print("Opening:", fits_file)

with fits.open(fits_file) as hdul:
    for i, hdu in enumerate(hdul):
        if hdu.data is not None:
            data = hdu.data
            header = hdu.header
            print("Data found in HDU:", i)
            print("Data min:", np.nanmin(data))
            print("Data max:", np.nanmax(data))
            print("HARPNUM:", header.get("HARPNUM"))
            print("T_REC:", header.get("T_REC"))
            break


Opening: C:\Users\Owner\sunpy\data\hmi.sharp_cea_720s.104.20100728_184800_TAI.Bp.fits
Data found in HDU: 1
Data min: -993.1
Data max: 1322.1100000000001
HARPNUM: 104
T_REC: 2010.07.28_18:48:00.000_TAI


In [101]:
from pathlib import Path

folder = Path(r"C:/Users/Owner/sunpy/data")
files = sorted(folder.glob("*.fits"))

print("Total FITS files:", len(files))
for f in files:
    print(f.name)


Total FITS files: 8972
hmi.sharp_cea_720s.104.20100728_184800_TAI.Bp.fits
hmi.sharp_cea_720s.104.20100728_184800_TAI.Br.fits
hmi.sharp_cea_720s.104.20100728_184800_TAI.Bt.fits
hmi.sharp_cea_720s.104.20100728_204800_TAI.Bp.fits
hmi.sharp_cea_720s.104.20100728_204800_TAI.Br.fits
hmi.sharp_cea_720s.104.20100728_204800_TAI.Bt.fits
hmi.sharp_cea_720s.104.20100728_224800_TAI.Bp.fits
hmi.sharp_cea_720s.104.20100728_224800_TAI.Br.fits
hmi.sharp_cea_720s.104.20100728_224800_TAI.Bt.fits
hmi.sharp_cea_720s.104.20100729_112400_TAI.Bp.fits
hmi.sharp_cea_720s.104.20100729_112400_TAI.Br.fits
hmi.sharp_cea_720s.104.20100729_112400_TAI.Bt.fits
hmi.sharp_cea_720s.104.20100729_132400_TAI.Bp.fits
hmi.sharp_cea_720s.104.20100729_132400_TAI.Br.fits
hmi.sharp_cea_720s.104.20100729_132400_TAI.Bt.fits
hmi.sharp_cea_720s.104.20100729_152400_TAI.Bp.fits
hmi.sharp_cea_720s.104.20100729_152400_TAI.Br.fits
hmi.sharp_cea_720s.104.20100729_152400_TAI.Bt.fits
hmi.sharp_cea_720s.104.20100730_021200_TAI.Bp.fits
hmi.shar

In [105]:
from sunpy.net import Fido, attrs as a
from pathlib import Path

outdir = Path(r"C:/Users/Owner/sunpy/data")
outdir.mkdir(parents=True, exist_ok=True)

harpnum = 6838
start_time = "2016-11-12T10:12:00"
end_time   = "2020-12-31T23:48:00"

query = Fido.search(
    a.Time(start_time, end_time),
    a.jsoc.Series("hmi.sharp_cea_720s"),
    a.jsoc.PrimeKey("HARPNUM", str(harpnum)),
    a.jsoc.Segment("Br"),
    a.jsoc.Segment("Bp"),
    a.jsoc.Segment("Bt"),
    a.jsoc.Notify("vishwastattala@gmail.com"),
)

Fido.fetch(query, path=str(outdir / "{file}"))


2026-01-29 21:34:53 - drms - INFO: Export request pending. [id=JSOC_20260130_000639, status=2]
2026-01-29 21:34:53 - drms - INFO: Waiting for 0 seconds...
2026-01-29 21:34:53 - drms - INFO: Export request pending. [id=JSOC_20260130_000639, status=1]
2026-01-29 21:34:53 - drms - INFO: Waiting for 5 seconds...
2026-01-29 21:34:59 - drms - INFO: Export request pending. [id=JSOC_20260130_000639, status=1]
2026-01-29 21:34:59 - drms - INFO: Waiting for 5 seconds...
2026-01-29 21:35:04 - drms - INFO: Export request pending. [id=JSOC_20260130_000639, status=1]
2026-01-29 21:35:04 - drms - INFO: Waiting for 5 seconds...
2026-01-29 21:35:09 - drms - INFO: Export request pending. [id=JSOC_20260130_000639, status=1]
2026-01-29 21:35:09 - drms - INFO: Waiting for 5 seconds...
2026-01-29 21:35:15 - drms - INFO: Export request pending. [id=JSOC_20260130_000639, status=1]
2026-01-29 21:35:15 - drms - INFO: Waiting for 5 seconds...
2026-01-29 21:35:20 - drms - INFO: Export request pending. [id=JSOC_20

INFO: 2814 URLs found for download. Full request totaling 734MB [sunpy.net.jsoc.jsoc]


Files Downloaded:   0%|                                                                     | 0/2814 [00:00<?, ?file/s]
hmi.sharp_cea_720s.6838.20161112_101200_TAI.Bp.fits:   0%|                                  | 0.00/274k [00:00<?, ?B/s]
hmi.sharp_cea_720s.6838.20161112_101200_TAI.Bp.fits:   0%|                         | 1.02k/274k [00:00<00:44, 6.13kB/s]
Files Downloaded:   0%|                                                             | 1/2814 [00:00<27:25,  1.71file/s]
hmi.sharp_cea_720s.6838.20161112_101200_TAI.Bt.fits:   0%|                                  | 0.00/288k [00:00<?, ?B/s]
hmi.sharp_cea_720s.6838.20161112_101200_TAI.Bt.fits:   0%|                         | 1.02k/288k [00:00<00:52, 5.44kB/s]
hmi.sharp_cea_720s.6838.20161112_101200_TAI.Bt.fits:  74%|████████████████████       | 214k/288k [00:00<00:00, 909kB/s]
Files Downloaded:   0%|                                                             | 2/2814 [00:01<24:06,  1.94file/s]
hmi.sharp_cea_720s.6838.20161112_101200_

5/0 files failed to download. Please check `.errors` for details


['C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.6838.20161112_101200_TAI.Bp.fits', 'C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.6838.20161112_101200_TAI.Bt.fits', 'C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.6838.20161112_101200_TAI.Br.fits', 'C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.6838.20161112_102400_TAI.Bp.fits', 'C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.6838.20161112_102400_TAI.Bt.fits', 'C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.6838.20161112_102400_TAI.Br.fits', 'C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.6838.20161112_103600_TAI.Bp.fits', 'C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.6838.20161112_103600_TAI.Bt.fits', 'C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.6838.20161112_103600_TAI.Br.fits', 'C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.6838.20161112_104800_TAI.Bp.fits', 'C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.6838.20161112_104800_TAI.Bt.fits', 'C:\\Users\\Owner\\sunpy\\data\\hmi.sharp_cea_720s.68

In [106]:
from pathlib import Path
import re

data_dir = Path(r"C:/Users/Owner/sunpy/data")

fits_files = list(data_dir.glob("*.fits"))

# Regex to extract HARPNUM and timestamp
pattern = re.compile(r"hmi\.sharp_cea_720s\.(\d+)\.(\d{8}_\d{6})_TAI\.(Br|Bp|Bt)\.fits")

harp6838 = []
other_harps = set()

for f in fits_files:
    m = pattern.match(f.name)
    if not m:
        continue
    harp, ts, seg = m.groups()
    if harp == "6838":
        harp6838.append((ts, seg))
    else:
        other_harps.add(harp)

# Count unique timestamps for HARPNUM 6838
unique_ts_6838 = sorted(set(ts for ts, _ in harp6838))

print("=== DIRECTORY AUDIT ===")
print(f"Total FITS files on disk: {len(fits_files)}")
print(f"Total HARPNUMs present: {len(other_harps) + 1}")
print()
print("=== HARPNUM 6838 ===")
print(f"Total FITS files for 6838: {len(harp6838)}")
print(f"Unique timestamps for 6838: {len(unique_ts_6838)}")
print(f"Expected FITS per timestamp: 3 (Br, Bp, Bt)")
print()
print("First timestamp for 6838:", unique_ts_6838[0])
print("Last timestamp for 6838:", unique_ts_6838[-1])

# Check for missing segments
missing = []
from collections import defaultdict
by_ts = defaultdict(set)
for ts, seg in harp6838:
    by_ts[ts].add(seg)

for ts, segs in by_ts.items():
    if segs != {"Br", "Bp", "Bt"}:
        missing.append((ts, segs))

print()
print("=== INTEGRITY CHECK ===")
if not missing:
    print("All timestamps have Br, Bp, Bt ✔")
else:
    print(f"Missing segments at {len(missing)} timestamps:")
    for ts, segs in missing[:10]:
        print(ts, "has", segs)


=== DIRECTORY AUDIT ===
Total FITS files on disk: 11781
Total HARPNUMs present: 220

=== HARPNUM 6838 ===
Total FITS files for 6838: 2809
Unique timestamps for 6838: 938
Expected FITS per timestamp: 3 (Br, Bp, Bt)

First timestamp for 6838: 20161112_101200
Last timestamp for 6838: 20161120_081200

=== INTEGRITY CHECK ===
Missing segments at 5 timestamps:
20161112_223600 has {'Bt', 'Bp'}
20161114_064800 has {'Bt', 'Br'}
20161115_181200 has {'Br', 'Bp'}
20161116_163600 has {'Bt', 'Bp'}
20161117_072400 has {'Bt', 'Bp'}


In [107]:
from pathlib import Path
import re
from collections import defaultdict

data_dir = Path(r"C:/Users/Owner/sunpy/data")

pattern = re.compile(
    r"hmi\.sharp_cea_720s\.(\d+)\.(\d{8}_\d{6})_TAI\.(Br|Bp|Bt)\.fits"
)

# harpnum -> timestamp -> segments
harp_map = defaultdict(lambda: defaultdict(set))

for f in data_dir.glob("*.fits"):
    m = pattern.match(f.name)
    if not m:
        continue
    harp, ts, seg = m.groups()
    harp_map[harp][ts].add(seg)

print("=== HARPNUM SUMMARY ===")
print(f"Total HARPNUMs on disk: {len(harp_map)}\n")

summary = []

for harp in sorted(harp_map, key=int):
    ts_map = harp_map[harp]
    n_ts = len(ts_map)
    n_complete = sum(1 for segs in ts_map.values() if segs == {"Br","Bp","Bt"})
    n_incomplete = n_ts - n_complete
    summary.append((harp, n_ts, n_complete, n_incomplete))

print(f"{'HARPNUM':>8} | {'TS':>5} | {'COMPLETE':>8} | {'INCOMPLETE':>10}")
print("-"*42)
for harp, n_ts, n_c, n_i in summary:
    print(f"{harp:>8} | {n_ts:>5} | {n_c:>8} | {n_i:>10}")

# Identify HARPNUMs with very small coverage (likely not yet extracted)
print("\n=== LIKELY NOT YET EXTRACTED (TS < 10) ===")
for harp, n_ts, _, _ in summary:
    if n_ts < 10:
        print(harp)


=== HARPNUM SUMMARY ===
Total HARPNUMs on disk: 220

 HARPNUM |    TS | COMPLETE | INCOMPLETE
------------------------------------------
       2 |     2 |        2 |          0
       8 |    15 |       15 |          0
      40 |     3 |        3 |          0
      43 |     3 |        3 |          0
      46 |    17 |       17 |          0
      51 |     3 |        3 |          0
      54 |     9 |        9 |          0
      57 |    12 |       12 |          0
      71 |     3 |        3 |          0
      83 |     2 |        2 |          0
      86 |    21 |       21 |          0
      92 |    28 |       28 |          0
     104 |    15 |       15 |          0
     114 |     3 |        3 |          0
     115 |     6 |        6 |          0
     131 |     9 |        9 |          0
     135 |     8 |        8 |          0
     145 |     9 |        9 |          0
     155 |     3 |        3 |          0
     156 |    12 |       12 |          0
     175 |    14 |       14 |          0
  

In [1]:
from pathlib import Path
import re
from collections import defaultdict

# Data directory
data_dir = Path(r"C:/Users/Owner/sunpy/data")

# Filename pattern
pattern = re.compile(
    r"hmi\.sharp_cea_720s\.(\d+)\.(\d{8}_\d{6})_TAI\.(Br|Bp|Bt)\.fits"
)

# harpnum -> timestamp -> segments
harp_map = defaultdict(lambda: defaultdict(set))

for f in data_dir.glob("*.fits"):
    m = pattern.match(f.name)
    if not m:
        continue
    harp, ts, seg = m.groups()
    harp_map[int(harp)][ts].add(seg)

print("=== HARPNUM COMPLETION AUDIT ===")
complete_harps = []

for harp in sorted(harp_map):
    ts_map = harp_map[harp]
    complete_ts = sum(1 for segs in ts_map.values() if segs == {"Br","Bp","Bt"})
    incomplete_ts = len(ts_map) - complete_ts

    if incomplete_ts == 0 and len(ts_map) > 50:
        complete_harps.append(harp)

    print(f"HARPNUM {harp}: TS={len(ts_map)} COMPLETE={complete_ts} INCOMPLETE={incomplete_ts}")

last_complete = max(complete_harps) if complete_harps else None

print("\n=== RESUME POINT ===")
print("Last fully completed HARPNUM:", last_complete)
print("Next HARPNUM to resume:", last_complete + 1 if last_complete else "UNKNOWN")


=== HARPNUM COMPLETION AUDIT ===
HARPNUM 2: TS=2 COMPLETE=2 INCOMPLETE=0
HARPNUM 8: TS=15 COMPLETE=15 INCOMPLETE=0
HARPNUM 40: TS=3 COMPLETE=3 INCOMPLETE=0
HARPNUM 43: TS=3 COMPLETE=3 INCOMPLETE=0
HARPNUM 46: TS=17 COMPLETE=17 INCOMPLETE=0
HARPNUM 51: TS=3 COMPLETE=3 INCOMPLETE=0
HARPNUM 54: TS=9 COMPLETE=9 INCOMPLETE=0
HARPNUM 57: TS=12 COMPLETE=12 INCOMPLETE=0
HARPNUM 71: TS=3 COMPLETE=3 INCOMPLETE=0
HARPNUM 83: TS=2 COMPLETE=2 INCOMPLETE=0
HARPNUM 86: TS=21 COMPLETE=21 INCOMPLETE=0
HARPNUM 92: TS=28 COMPLETE=28 INCOMPLETE=0
HARPNUM 104: TS=15 COMPLETE=15 INCOMPLETE=0
HARPNUM 114: TS=3 COMPLETE=3 INCOMPLETE=0
HARPNUM 115: TS=6 COMPLETE=6 INCOMPLETE=0
HARPNUM 131: TS=9 COMPLETE=9 INCOMPLETE=0
HARPNUM 135: TS=8 COMPLETE=8 INCOMPLETE=0
HARPNUM 145: TS=9 COMPLETE=9 INCOMPLETE=0
HARPNUM 155: TS=3 COMPLETE=3 INCOMPLETE=0
HARPNUM 156: TS=12 COMPLETE=12 INCOMPLETE=0
HARPNUM 175: TS=14 COMPLETE=14 INCOMPLETE=0
HARPNUM 182: TS=3 COMPLETE=3 INCOMPLETE=0
HARPNUM 185: TS=9 COMPLETE=9 INCOMPLETE=0

In [2]:
from pathlib import Path
import re
from collections import defaultdict

data_dir = Path(r"C:/Users/Owner/sunpy/data")

pattern = re.compile(
    r"hmi\.sharp_cea_720s\.(\d+)\.(\d{8}_\d{6})_TAI\.(Br|Bp|Bt)\.fits"
)

harp_ts = defaultdict(lambda: defaultdict(set))

for f in data_dir.glob("*.fits"):
    m = pattern.match(f.name)
    if not m:
        continue
    harp, ts, seg = m.groups()
    harp_ts[int(harp)][ts].add(seg)

missing = []

print("\n=== MISSING FILES ===")
for harp in sorted(harp_ts):
    if harp < 6838:
        continue
    for ts, segs in harp_ts[harp].items():
        for seg in {"Br", "Bp", "Bt"} - segs:
            missing.append((harp, ts, seg))
            print(f"HARPNUM {harp}  {ts}  missing {seg}")

print(f"\nTotal missing files: {len(missing)}")



=== MISSING FILES ===
HARPNUM 6838  20161112_223600  missing Br
HARPNUM 6838  20161114_064800  missing Bp
HARPNUM 6838  20161115_181200  missing Bt
HARPNUM 6838  20161116_163600  missing Br
HARPNUM 6838  20161117_072400  missing Br
HARPNUM 6839  20161111_084800  missing Br
HARPNUM 6839  20161111_173600  missing Br
HARPNUM 6839  20161112_013600  missing Bp
HARPNUM 6840  20161112_164800  missing Bt
HARPNUM 6841  20161113_140000  missing Br
HARPNUM 6841  20161114_161200  missing Bt
HARPNUM 6841  20161114_162400  missing Br
HARPNUM 6841  20161115_054800  missing Br
HARPNUM 6841  20161116_124800  missing Bp
HARPNUM 6842  20161114_052400  missing Br
HARPNUM 6842  20161114_064800  missing Bt
HARPNUM 6842  20161115_040000  missing Bp
HARPNUM 6842  20161116_033600  missing Bt
HARPNUM 6842  20161116_063600  missing Br
HARPNUM 6842  20161116_132400  missing Br
HARPNUM 6842  20161116_172400  missing Bp
HARPNUM 6843  20161114_113600  missing Bp
HARPNUM 6843  20161114_123600  missing Bp
HARPNUM 684

In [17]:
from pathlib import Path
import re

# Use a raw string for the path to handle backslashes correctly
batch_dir = Path(r"C:/Users/Owner/Onedrive/Desktop/solar_proj/jsoc_batches")

# Corrected pattern:
# 1. Escaped brackets: \[ \]
# 2. No spaces between segments: ]\[
record_pattern = re.compile(r"hmi\.sharp_cea_720s\[(\d+)\]\[(\d{8}_\d{6})\]")

required_records = set()

# Check if directory exists to avoid silent failure
if batch_dir.exists():
    for f in batch_dir.glob("cnn_batch_*_newline.txt"):
        text = f.read_text()
        for m in record_pattern.finditer(text):
            harp, ts = m.groups()
            required_records.add((int(harp), ts))

    print(f"Total required records: {len(required_records)}")
else:
    print(f"Directory not found: {batch_dir}")

Total required records: 0


In [23]:
from pathlib import Path
import re
from collections import Counter, defaultdict

data_dir = Path(r"C:/Users/Owner/sunpy/data")

pat = re.compile(r"hmi\.sharp_cea_720s\.(\d+)\.(\d{8}_\d{6}_TAI)\.(Br|Bp|Bt)\.fits")

recs = defaultdict(set)     # (harp, ts) -> set(segs)
harp_counts = Counter()     # harp -> number of unique (harp,ts)
all_pairs = set()

for f in data_dir.glob("*.fits"):
    m = pat.match(f.name)
    if not m:
        continue
    harp, ts, seg = m.groups()
    recs[(harp, ts)].add(seg)
    all_pairs.add((harp, ts))

for (harp, ts) in all_pairs:
    harp_counts[int(harp)] += 1

harps = sorted(harp_counts.keys())
print("HARPNUM range on disk:", (harps[0] if harps else None), "to", (harps[-1] if harps else None))
print("Unique recordsets on disk:", len(all_pairs))

# Most recent timestamp on disk (lexicographic works with YYYYMMDD_HHMMSS_TAI)
most_recent = max(all_pairs, key=lambda x: x[1]) if all_pairs else None
print("Most recent (HARPNUM, TS) on disk:", most_recent)

# Top HARPNUMs by number of timestamps present
print("\nTop 20 HARPNUMs by recordset count:")
for harp, n in harp_counts.most_common(20):
    print(harp, n)

# Incomplete recordsets
incomplete = [(k, v) for k, v in recs.items() if len(v) != 3]
print("\nIncomplete recordsets:", len(incomplete))
for (harp, ts), segs in incomplete[:20]:
    missing = sorted({"Br","Bp","Bt"} - set(segs))
    print("HARPNUM", harp, "TS", ts, "have", sorted(segs), "missing", missing)


HARPNUM range on disk: 2 to 6846
Unique recordsets on disk: 7499
Most recent (HARPNUM, TS) on disk: ('6845', '20161123_020000_TAI')

Top 20 HARPNUMs by recordset count:
6845 1040
6838 938
6843 896
6842 535
6841 393
6839 368
6846 190
6840 148
1807 110
5983 83
393 72
6063 71
6483 70
377 68
5956 63
750 57
6015 55
856 43
892 41
637 39

Incomplete recordsets: 1
HARPNUM 6100 TS 20151122_103600_TAI have ['Br', 'Bt'] missing ['Bp']


In [26]:
from pathlib import Path
import re

batch_dir = Path("jsoc_batches")

# Corrected Regex:
# 1. Escaped the brackets with \
# 2. Removed the accidental whitespace/newlines
pat = re.compile(r"hmi\.sharp_cea_720s\[(\d+)\]")

harps = set()

# Ensure the directory exists to avoid errors
if batch_dir.exists():
    for f in batch_dir.glob("*_newline.txt"):
        text = f.read_text()
        for m in pat.finditer(text):
            harps.add(int(m.group(1)))

# Safety check: min() and max() fail on an empty set
if harps:
    print(f"Min HARPNUM:   {min(harps)}")
    print(f"Max HARPNUM:   {max(harps)}")
    print(f"Total HARPNUMs: {len(harps)}")
else:
    print("No HARPNUMs found. Check your file paths or pattern.")

Min HARPNUM:   2
Max HARPNUM:   7493
Total HARPNUMs: 314
